In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

import os

import warnings
warnings.filterwarnings('ignore')

DATA_DIR = '../data'

In [20]:
# load files
train = pd.read_csv(os.path.join(DATA_DIR, 'gender_age_train.csv'), dtype={'device_id': np.str})
test = pd.read_csv(os.path.join(DATA_DIR, 'gender_age_test.csv'), dtype={'device_id': np.str})

In [3]:
# load app labels
app_labels = pd.read_csv(os.path.join(DATA_DIR, 'app_labels.csv'), dtype={'device_id': np.str})

In [6]:
# concatenate all different labels associated with an application

app_labels = app_labels.groupby('app_id')['label_id'].apply(lambda x: ' '.join(str(s) for s in x))

In [7]:
# load app events

app_events = pd.read_csv(os.path.join(DATA_DIR, 'app_events.csv'))

In [10]:
# create a similar mapping of app labels in events collection
app_events['app_labels'] = app_events.app_id.map(app_labels)

In [12]:
# group by event id and map all the app labels agains that event id.
app_events = app_events.groupby('event_id')['app_labels'].apply(lambda x: ' '.join(str(s) for s in x))

In [13]:
# load events
events = pd.read_csv(os.path.join(DATA_DIR, 'events.csv'), dtype={'device_id': np.str})

In [14]:
events['app_labels'] = events.event_id.map(app_events)

In [15]:
events = events.groupby('device_id')['app_labels'].apply(lambda x: ' '.join(str(s) for s in x))

In [16]:
# phone brands
phone_brand = pd.read_csv(os.path.join(DATA_DIR, 'phone_brand_device_model.csv'), dtype={'device_id': np.str})
phone_brand.drop_duplicates('device_id', keep='first', inplace=True)

In [21]:
train['app_labels'] = train.device_id.map(events)
train = pd.merge(train, phone_brand, on='device_id', how='left', left_index=True)

test['app_labels'] = test.device_id.map(events)
test = pd.merge(test, phone_brand, on='device_id', how='left', left_index=True)

In [24]:
train.to_csv('../processed/train_processed.csv', index=False)
test.to_csv('../processed/test_processed.csv', index=False)